In [189]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os, glob
import json

# define project directories 
root = os.path.abspath(os.getcwd())

In [39]:
#read in charity data
cc_df = pd.read_csv("registered-charity_20220724.csv")
cc_ch = cc_df[~cc_df["companyNumber"].isna()]

C:\Users\LISA~1.HOR\AppData\Local\Temp/ipykernel_51456/188355128.py:2: DtypeWarning: Columns (2,3,7) have mixed types. Specify dtype option on import or set low_memory=False.
  cc_df = pd.read_csv("registered-charity_20220724.csv")


In [190]:
f = open('codes.json') 
sic_dict = json.load(f) 

In [51]:
# read in gender pay gap data and merge files
all_files = glob.glob(os.path.join(root, "UK Gender Pay Gap Data*.csv"))

all_df = []
for f in all_files:
    df = pd.read_csv(f, sep=',')
    df['file'] = f.split('\\')[-1][-16:-4]
    df = pd.merge(df, cc_ch, left_on="CompanyNumber", right_on="companyNumber", how='left')
    all_df.append(df)
    
pay_df = pd.concat(all_df, ignore_index=True, sort=True)

# Explore

In [81]:
pay_df["SicCodes"]

0                85310
1                47730
2                78300
3                93110
4        56210,\n70229
             ...      
48700            63120
48701    78300,\n82990
48702            82990
48703            64999
48704            10110
Name: SicCodes, Length: 48705, dtype: object

### SIC codes

In [172]:
#clean up SIC codes a bit
pay_df["SIC_clean"] = pay_df["SicCodes"].str.replace("1,", "").str.replace("\n", "")
pay_df["SIC_clean"] = pay_df["SIC_clean"].fillna("")
pay_df["SIC_clean"] = pay_df["SIC_clean"].str.split(",")
pay_df["SIC_count"] = pay_df["SIC_clean"].str.len()

#map against section and industry name from dictionary

section_all = []
name_all = []

for i in range(len(pay_df["SIC_clean"])):
    section = []
    name = []
    for k in range(len(pay_df["SIC_clean"][i])):
        try:
            section.append(sic_dict[pay_df["SIC_clean"][i][k]]["section"])
            name.append(sic_dict[pay_df["SIC_clean"][i][k]]["industry"])
        except KeyError:
            pass
    section_all.append(section)
    name_all.append(name)

#add to dataframe
pay_df["SIC_sections"] = section_all
pay_df["SIC_names"] = name_all

#select first one as the main SIC
pay_df["SIC_section"] = pay_df["SIC_sections"].str[0]
pay_df["SIC_name"] = pay_df["SIC_names"].str[0]

In [73]:
#Tech specific codes
tech = [ "58290", "62011", "62012", "62020", "62030", "62090", "63110", "63120"]

for code in tech:
    pay_df.loc[pay_df["SicCodes"].str.contains(code, na=False), "Sector"] = "Tech"

In [184]:
pivot = pay_df.groupby(["file", "SIC_section"])[['DiffMeanHourlyPercent']].mean()
pivot.unstack().transpose()

file                               2017 to 2018  2018 to 2019  2019 to 2020  \
                      SIC_section                                             
DiffMeanHourlyPercent C               13.307916     12.811722     12.125608   
                      D               15.501786     13.273770     13.992453   
                      E                6.908451      7.512500      7.360000   
                      F               22.047797     22.114887     20.239713   
                      G               14.852436     14.868947     14.804044   
                      H               10.209409     10.637895     10.732314   
                      I                7.989861      7.018304      7.525446   
                      J               19.437890     19.380045     18.741613   
                      K               26.184715     26.401759     26.891277   
                      L               16.328319     16.632231     17.573684   
                      M               18.165816     18.756421     19.787174   
                      N               11.490768     10.620644     10.483079   
                      O                8.088177      7.801486      7.382167   
                      P               15.825763     16.020751     15.773064   
                      Q                9.633615     10.680874     11.510155   
                      R               20.874729     20.642143     15.843590   
                      S               12.577778     12.533810     11.567669   
                      T                3.133333     -0.375000     -0.600000   
                      U                9.475000      6.442857     12.700000   

file                               2020 to 2021  2021 to 2022  
                      SIC_section                              
DiffMeanHourlyPercent C               12.733934     10.689612  
                      D               15.060317     15.270968  
                      E                6.768354      5.539241  
                      F               21.634219     20.053169  
                      G               14.158817     14.039179  
                      H                9.299213      9.155467  
                      I                8.806043      8.065885  
                      J               18.142857     17.821158  
                      K               25.257834     24.949884  
                      L               15.801681     16.120870  
                      M               18.467429     18.184746  
                      N               10.474196      9.804222  
                      O                6.750099      6.425502  
                      P               15.906882     16.049583  
                      Q               10.115601      9.910315  
                      R               18.841393     22.324107  
                      S               12.968932     13.184080  
                      T               -1.700000     -1.466667  
                      U               13.890000      4.425000

In [185]:
pivot = pay_df.groupby(["file", "SIC_section"])[[ 'FemaleTopQuartile']].mean()
pivot.unstack().transpose()

file                           2017 to 2018  2018 to 2019  2019 to 2020  \
                  SIC_section                                             
FemaleTopQuartile C               19.048394     19.519826     20.282735   
                  D               17.682143     17.481967     16.688679   
                  E               20.263380     20.090278     20.074000   
                  F               12.415593     12.979288     13.910526   
                  G               36.819703     36.785579     33.155882   
                  H               15.588710     16.298158     16.560262   
                  I               44.610934     45.411561     48.197321   
                  J               24.732854     24.711435     24.862903   
                  K               29.687565     30.647236     30.511526   
                  L               36.611504     35.648760     34.325000   
                  M               32.960142     33.781530     33.025870   
                  N               37.691458     38.053427     36.751179   
                  O               47.835222     48.356900     48.417381   
                  P               61.492748     61.807344     62.616386   
                  Q               73.147037     72.517268     71.798795   
                  R               38.035379     39.161071     42.056410   
                  S               43.375845     44.418571     43.666917   
                  T               50.666667     50.200000     36.933333   
                  U               42.050000     43.900000     32.100000   

file                           2020 to 2021  2021 to 2022  
                  SIC_section                              
FemaleTopQuartile C               20.659801     20.807109  
                  D               17.073016     18.798387  
                  E               21.202564     20.836709  
                  F               14.076351     14.095745  
                  G               34.727182     35.249434  
                  H               17.045867     17.335150  
                  I               44.123256     45.302000  
                  J               25.270789     25.292121  
                  K               31.207407     30.673782  
                  L               38.368103     36.620000  
                  M               33.663873     34.577364  
                  N               37.769258     37.892069  
                  O               49.018091     48.978659  
                  P               63.627174     63.589509  
                  Q               72.579935     72.278571  
                  R               39.039744     35.267299  
                  S               42.686567     43.567677  
                  T               37.900000     40.433333  
                  U               21.920000     16.975000

In [74]:
pay_df["Sector"].value_counts()

Tech    1438
Name: Sector, dtype: int64

In [70]:
pay_df['SicCodes']

0                85310
1                47730
2                78300
3                93110
4        56210,\n70229
             ...      
48700            63120
48701    78300,\n82990
48702            82990
48703            64999
48704            10110
Name: SicCodes, Length: 48705, dtype: object

In [79]:
pay_df[pay_df["Sector"]=="Tech"].groupby("file")[['DiffMeanHourlyPercent','DiffMedianHourlyPercent', 
                                                      'DiffMeanBonusPercent', 'DiffMedianBonusPercent',
                                                  'FemaleLowerQuartile',
                         'FemaleTopQuartile', 'FemaleUpperMiddleQuartile']].mean()

,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,FemaleLowerQuartile,FemaleTopQuartile,FemaleUpperMiddleQuartile
file,,,,,,,
2017 to 2018,20.462264,18.886415,39.387925,20.378491,40.583019,19.705660,24.775849
2018 to 2019,20.932997,19.630640,37.484642,22.364505,40.732323,19.847475,25.464646
2019 to 2020,20.753917,19.647465,40.579167,28.469444,41.556682,20.483410,26.420276
2020 to 2021,19.436875,19.005000,38.836102,16.565815,41.700637,21.429618,26.019745
2021 to 2022,18.963127,17.871976,26.193452,-2.938988,40.948512,21.822321,26.870833


In [78]:
pay_df[pay_df["Sector"]=="Tech"].groupby("file")[['DiffMeanHourlyPercent','DiffMedianHourlyPercent', 
                                              
                                                  'FemaleLowerQuartile',
                         'FemaleTopQuartile', 'FemaleUpperMiddleQuartile']].count()

,DiffMeanHourlyPercent,DiffMedianHourlyPercent,DiffMeanBonusPercent,DiffMedianBonusPercent,FemaleLowerQuartile,FemaleTopQuartile,FemaleUpperMiddleQuartile
file,,,,,,,
2017 to 2018,265,265,265,265,265,265,265
2018 to 2019,297,297,293,293,297,297,297
2019 to 2020,217,217,216,216,217,217,217
2020 to 2021,320,320,313,313,314,314,314
2021 to 2022,339,339,336,336,336,336,336


In [64]:
charity.groupby("file")[['DiffMeanHourlyPercent','DiffMedianHourlyPercent', 'FemaleLowerQuartile',
                         'FemaleTopQuartile', 'FemaleUpperMiddleQuartile']].mean()

,DiffMeanHourlyPercent,DiffMedianHourlyPercent,FemaleLowerQuartile,FemaleTopQuartile,FemaleUpperMiddleQuartile
file,,,,,
2017 to 2018,8.044463,5.653388,70.561983,62.682149,68.518347
2018 to 2019,7.946166,6.926591,70.845024,62.632137,68.232953
2019 to 2020,8.810000,7.696098,71.658293,62.901220,68.532927
2020 to 2021,8.134647,6.370936,70.561258,62.734106,67.744702
2021 to 2022,8.456126,6.999503,70.872148,62.598826,67.552517


In [58]:
charity.columns

Index(['Address', 'CompanyLinkToGPGInfo', 'CompanyNumber', 'CurrentName',
       'DateSubmitted', 'DiffMeanBonusPercent', 'DiffMeanHourlyPercent',
       'DiffMedianBonusPercent', 'DiffMedianHourlyPercent', 'DueDate',
       'EmployerId', 'EmployerName', 'EmployerSize', 'FemaleBonusPercent',
       'FemaleLowerMiddleQuartile', 'FemaleLowerQuartile', 'FemaleTopQuartile',
       'FemaleUpperMiddleQuartile', 'MaleBonusPercent',
       'MaleLowerMiddleQuartile', 'MaleLowerQuartile', 'MaleTopQuartile',
       'MaleUpperMiddleQuartile', 'PostCode', 'ResponsiblePerson', 'SicCodes',
       'SubmittedAfterTheDeadline', 'active', 'charityNumber', 'companyNumber',
       'dateModified', 'dateRegistered', 'dateRemoved', 'file', 'id',
       'latestIncome', 'latestIncomeDate', 'linked_orgs', 'name', 'orgIDs',
       'organisationType', 'organisationTypePrimary', 'postalCode', 'source',
       'url'],
      dtype='object')

In [57]:
charity = pay_df[~pay_df["charityNumber"].isna()]
charity

,Address,CompanyLinkToGPGInfo,CompanyNumber,CurrentName,DateSubmitted,DiffMeanBonusPercent,DiffMeanHourlyPercent,DiffMedianBonusPercent,DiffMedianHourlyPercent,DueDate,...,latestIncome,latestIncomeDate,linked_orgs,name,orgIDs,organisationType,organisationTypePrimary,postalCode,source,url
0,"Bryanston House, Blandford, Dorset, DT11 0PX",https://www.bryanston.co.uk/employment,00226143,BRYANSTON SCHOOL INCORPORATED,2018/03/27 11:42:49,0.0,18.0,0.0,28.2,2018/04/05 00:00:00,...,23767876.0,2021-08-31,"['GB-CHC-306210', 'GB-COH-00226143', 'GB-EDU-1...",BRYANSTON SCHOOL INCORPORATED,"['GB-CHC-306210', 'GB-COH-00226143']","['registered-charity', 'registered-charity-eng...",Registered Charity,DT11 0PX,ccew,www.bryanston.co.uk
3,"Trinity Sports And Leisure, Chilton Street, Br...",https://www.1610.org.uk/gender-pay-gap/,06727055,1610 LIMITED,2018/04/12 17:03:26,-47.0,-22.0,-67.0,-34.0,2018/04/05 00:00:00,...,3502797.0,2021-03-31,"['GB-CHC-1130010', 'GB-COH-06727055']",1610 LIMITED,"['GB-CHC-1130010', 'GB-COH-06727055']","['registered-charity', 'registered-charity-eng...",Registered Charity,TA6 3JA,ccew,www.1610.org.uk
19,"23 Carbrook Hall Road, Sheffield, England, S9 2EH",NaN,07923816,7 HILLS LEISURE TRUST,2018/03/23 17:27:15,-100.0,-13.6,-100.0,-15.7,2018/04/05 00:00:00,...,47000.0,2021-03-31,"['GB-CHC-1150711', 'GB-COH-07923816']",7 HILLS LEISURE TRUST,"['GB-CHC-1150711', 'GB-COH-07923816']","['registered-charity', 'registered-charity-eng...",Registered Charity,S9 2EH,ccew,NaN
66,"Haverhill Leisure Centre, Ehringshausen Way, H...",NaN,05207019,ABBEYCROFT LEISURE,2018/04/01 11:06:37,0.0,-5.4,0.0,-8.4,2018/04/05 00:00:00,...,6027305.0,2021-03-31,"['GB-CHC-1117138', 'GB-COH-05207019']",ABBEYCROFT LEISURE,"['GB-CHC-1117138', 'GB-COH-05207019']","['registered-charity', 'registered-charity-eng...",Registered Charity,CB9 0ER,ccew,www.acleisure.com
67,"St Peter's House, 2 Bricket Road, St. Albans, ...",https://www.abbeyfield.com/about-abbeyfield/,00574816,ABBEYFIELD SOCIETY (THE),2018/04/04 12:05:33,0.0,17.0,0.0,10.0,2018/04/05 00:00:00,...,53561000.0,2021-03-31,"['GB-CHC-200719', 'GB-COH-00574816', 'GB-SHPE-...",THE ABBEYFIELD SOCIETY,"['GB-CHC-200719', 'GB-COH-00574816']","['registered-charity', 'registered-charity-eng...",Registered Charity,AL1 3JW,ccew,www.abbeyfield.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48638,"Trevelyan House, Dimple Road, Matlock, Derbysh...",https://www.yha.org.uk/our-policies/compliance...,00282555,YHA (ENGLAND AND WALES),2022/02/22 08:09:11,NaN,17.4,NaN,24.8,2022/04/05 00:00:00,...,26263000.0,2021-02-28,"['GB-CHC-301657', 'GB-CHC-306122', 'GB-COH-002...",YHA (ENGLAND AND WALES),"['GB-CHC-306122', 'GB-COH-00282555']","['registered-charity', 'registered-charity-eng...",Registered Charity,DE4 3YH,ccew,www.yha.org.uk
48640,"REED HOUSE, 47 Church Road, Hove, East Sussex,...",NaN,03853734,YMCA DOWNSLINK GROUP,2022/03/28 16:02:14,21.2,-10.9,32.4,-11.9,2022/04/05 00:00:00,...,16203951.0,2021-03-31,"['GB-CHC-1039278', 'GB-CHC-1073778', 'GB-CHC-1...",YMCA DOWNSLINK GROUP,"['GB-CHC-1079570', 'GB-COH-03853734']","['registered-charity', 'registered-charity-eng...",Registered Charity,BN3 2BE,ccew,www.ymcadlg.org
48641,"Fairthorne Manor Botley Road, Curbridge, South...",https://ymca-fg.org/?s=gender+pay+gap,04336719,YMCA FAIRTHORNE GROUP,2022/02/14 09:30:10,NaN,8.0,NaN,0.0,2022/04/05 00:00:00,...,12783000.0,2021-04-30,"['GB-CHC-1090981', 'GB-CHC-1096885', 'GB-COH-0...",YMCA FAIRTHORNE GROUP,"['GB-CHC-1090981', 'GB-COH-04336719']","['registered-charity', 'registered-charity-eng...",Registered Charity,SO30 2GH,ccew,www.ymca-fg.org
48642,"49 Victoria Road, Surbiton, England, KT6 4NG",NaN,02971930,YMCA ST PAUL'S GROUP,2022/02/22 09:39:57,NaN,-0.3,NaN,4.5,2022/04/05 00:00:00,...,23722676.0,2021-03-31,"['GB-CHC-1041923', 'GB-CHC-1045613', 'GB-CHC-1...",YMCA St Paul's Group,"['GB-CHC-1041923', 'GB-COH-02971930']","['registered-charity', 'registered-charity-eng...",Registered Charity,KT6 